# Optimizing a Mutual Fund Portfolio using NSE Stocks

In [1]:
# Importing essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure visualization settings
sns.set(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 11

### Load and preview the dataset
The dataset contains daily stock data for companies listed on the Nairobi Securities Exchange from January to October 2024.
This section loads the data, checks its structure, and performs an initial inspection to confirm that the columns and data types are correctly formatted.

In [2]:
# Load dataset
df = pd.read_csv("NSE_data_all_stocks_2024_jan_to_oct.csv")

# Display the first few records
df.head()


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,2-Jan-24,EGAD,Eaagads Ltd,10.35,14.5,12.8,12.8,12.8,13.95,-1.15,8.24%,100,-
1,2-Jan-24,KUKZ,Kakuzi Plc,342,440,385,385,385,385,-,-,-,-
2,2-Jan-24,KAPC,Kapchorua Tea Kenya Plc,207,280,215,215,215,215,-,-,-,-
3,2-Jan-24,LIMT,Limuru Tea Plc,365,380,380,380,380,380,-,-,-,-
4,2-Jan-24,SASN,Sasini Plc,15.1,22,20,20,20,20,-,-,"3,300.00",-


In [3]:
# Shape and basic information
print(f"Dataset shape: {df.shape}")
df.info()


Dataset shape: (15258, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15258 entries, 0 to 15257
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            15258 non-null  object
 1   Code            15258 non-null  object
 2   Name            15258 non-null  object
 3   12m Low         15258 non-null  object
 4   12m High        15258 non-null  object
 5   Day Low         15258 non-null  object
 6   Day High        15258 non-null  object
 7   Day Price       15258 non-null  object
 8   Previous        15258 non-null  object
 9   Change          15258 non-null  object
 10  Change%         15258 non-null  object
 11  Volume          15258 non-null  object
 12  Adjusted Price  15258 non-null  object
dtypes: object(13)
memory usage: 1.5+ MB


In [ ]:
# Check for missing values and duplicated rows
df.isna().sum()
df.duplicated().sum()


0

### Data Cleaning and formatting 
The dataset contains several numeric columns stored as text and placeholders like '-' instead of null values.
This section cleans the data by replacing invalid symbols, converting data types, and sorting the dataset for consistency.

In [5]:
# Replace placeholder dashes with NaN
df.replace('-', np.nan, inplace=True)

# Convert numeric columns to proper floats
num_cols = ['12m Low', '12m High', 'Day Low', 'Day High',
            'Day Price', 'Previous', 'Change', 'Volume']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')

# Drop rows without valid Day Price
df.dropna(subset=['Day Price'], inplace=True)

# Sort by stock code and date
df.sort_values(['Code', 'Date'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Confirm data types and new structure
df.info()
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13812 entries, 0 to 13811
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            13812 non-null  datetime64[ns]
 1   Code            13812 non-null  object        
 2   Name            13812 non-null  object        
 3   12m Low         13812 non-null  float64       
 4   12m High        13650 non-null  float64       
 5   Day Low         13812 non-null  float64       
 6   Day High        13812 non-null  float64       
 7   Day Price       13812 non-null  float64       
 8   Previous        13806 non-null  float64       
 9   Change          8225 non-null   float64       
 10  Change%         8219 non-null   object        
 11  Volume          1898 non-null   float64       
 12  Adjusted Price  0 non-null      float64       
dtypes: datetime64[ns](1), float64(9), object(3)
memory usage: 1.4+ MB


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,2024-01-02,ABSA,ABSA Bank Kenya Plc,10.55,16.7,11.45,11.75,11.55,11.45,0.1,0.87%,NaN,NaN
1,2024-01-03,ABSA,ABSA Bank Kenya Plc,10.55,16.7,11.40,11.70,11.45,11.55,-0.1,0.87%,NaN,NaN
2,2024-01-04,ABSA,ABSA Bank Kenya Plc,10.55,16.7,11.40,11.65,11.55,11.45,0.1,0.87%,NaN,NaN
3,2024-01-05,ABSA,ABSA Bank Kenya Plc,10.55,16.7,11.55,11.75,11.65,11.55,0.1,0.87%,NaN,NaN
4,2024-01-08,ABSA,ABSA Bank Kenya Plc,10.55,16.7,11.55,11.75,11.65,11.55,0.1,0.87%,NaN,NaN


Some columns such as Adjusted Price, Change, and Change% contain significant missing data or duplicate information.

These are dropped to retain only relevant features for calculating ROI and volatility.Some columns such as Adjusted Price, Change, and Change% contain significant missing data or duplicate information.
These are dropped to retain only relevant features for calculating ROI and volatility.

In [6]:
# Drop unnecessary columns
df.drop(columns=['Adjusted Price', 'Change', 'Change%'], inplace=True)

# Final check for null values
df.isna().sum()


Date             0
Code             0
Name             0
12m Low          0
12m High       162
Day Low          0
Day High         0
Day Price        0
Previous         6
Volume       11914
dtype: int64

## Calculating ROI and Volatility


To identify the best-performing stocks for the mutual fund plan,
ROI (Return on Investment) and volatility are calculated for each stock.

ROI measures total return over the analysis period.

Volatility represents the stock’s risk, calculated as the standard deviation of daily returns.

The goal is to select stocks with high ROI and low volatility, balancing growth and stability.